<a href="https://colab.research.google.com/github/pelinbalci/LLM_Notebooks/blob/main/t5__question_answer_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00


In [4]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00


In [1]:
# Example of using T5 with Hugging Face Transformers library
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# For embeddings
inputs = tokenizer("How can I make a cake", return_tensors="pt", padding=True, truncation=True)
embeddings = model.encoder(inputs.input_ids)

# For text generation
outputs = model.generate(inputs.input_ids)
generated_text = tokenizer.decode(outputs[0])


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [3]:
import pandas as pd

# initialize data of lists.
data = {'Question': ['How to make a cake', 'How to make omette', 'how can I make a soup', 'best way of making cake'],
		'Answer': ["Put Eggs and oil in a cup and cook them in oven for 30 minutes", "with eggs and milk. Cook for 3 minutes", "Put tomato and peper and boil it", "heat the oven to 200 degrees. Use eggs oil and milk"]}

# Create DataFrame
df = pd.DataFrame(data)
df

,Question,Answer
0,How to make a cake,Put Eggs and oil in a cup and cook them in ove...
1,How to make omette,with eggs and milk. Cook for 3 minutes
2,how can I make a soup,Put tomato and peper and boil it
3,best way of making cake,heat the oven to 200 degrees. Use eggs oil and...


In [15]:
# Embeddings of all data.Question

query = tokenizer("How can I make a cake", return_tensors="pt", padding=True, truncation=True)
embeddings = model.encoder(query.input_ids)
embeddings.last_hidden_state.detach().numpy().shape

(1, 8, 512)

In [18]:
embeddings

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.0656,  0.0444,  0.1625,  ..., -0.0642,  0.0443, -0.0468],
         [ 0.1966, -0.0561, -0.1466,  ...,  0.1691, -0.1165, -0.1255],
         [-0.0472, -0.0491, -0.0068,  ...,  0.1455, -0.1416, -0.0607],
         ...,
         [ 0.0125,  0.2147,  0.0928,  ..., -0.0681, -0.0597, -0.0282],
         [ 0.1210,  0.1537, -0.0239,  ..., -0.1451, -0.3125, -0.0690],
         [ 0.2413,  0.1123, -0.0243,  ...,  0.0117,  0.1676,  0.0970]]],
       grad_fn=<MulBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [6]:
def get_embeddings(sentences:list):
  list_sentence_embeddings =[]
  for sen in sentences:
    sentence_embeddings = model.encode(sen)
    list_sentence_embeddings.append(sentence_embeddings)
  df = pd.DataFrame({"sentences": sentences, "embedding": list_sentence_embeddings})
  return df

# Sentence Transofrmers Embeddings & Search

In [ ]:
pip install -U sentence-transformers

In [19]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/sentence-t5-base')

In [20]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = embedding_model.encode(sentences)
print(embeddings)

[[-0.0090485   0.0191336   0.02657794 ... -0.00876412 -0.05602207
  -0.02163093]
 [-0.00785423  0.03018175  0.03129675 ... -0.01246876 -0.06240674
  -0.00590968]]


In [21]:
import pandas as pd

# initialize data of lists.
data = {'Question':
 ['How to make a cake',
  "How can I make a cake in 10 minutes?",
  'How to make omette',
  'how can I make a soup',
  'best way of making cake',
  "What is tips for making cakes?",
  "I want to make a cake with lemon"],
		'Answer':
["Put Eggs and oil in a cup and cook them in oven for 30 minutes",
 "You can buy a cake mix from store and use a microwave",
 "with eggs and milk. Cook for 3 minutes",
 "Put tomato and peper and boil it",
 "heat the oven to 200 degrees. Use eggs oil and milk",
 "You need to heat the oven before prepairign the ingredients.",
 "Put sugar, lemon, egg and oil in a cup and cook for 30 minutes. "]}

# Create DataFrame
df = pd.DataFrame(data)
df

,Question,Answer
0,How to make a cake,Put Eggs and oil in a cup and cook them in ove...
1,How can I make a cake in 10 minutes?,You can buy a cake mix from store and use a mi...
2,How to make omette,with eggs and milk. Cook for 3 minutes
3,how can I make a soup,Put tomato and peper and boil it
4,best way of making cake,heat the oven to 200 degrees. Use eggs oil and...
5,What is tips for making cakes?,You need to heat the oven before prepairign th...
6,I want to make a cake with lemon,"Put sugar, lemon, egg and oil in a cup and coo..."


In [22]:
list_sentence_embeddings =[]
for sen in list(df.Question):
  sentence_embeddings = embedding_model.encode(sen)
  list_sentence_embeddings.append(sentence_embeddings)
df["embeddings"] = list_sentence_embeddings
df

,Question,Answer,embeddings
0,How to make a cake,Put Eggs and oil in a cup and cook them in ove...,"[-0.010374778, -0.041325103, 0.035711173, 0.02..."
1,How can I make a cake in 10 minutes?,You can buy a cake mix from store and use a mi...,"[-0.016724477, -0.01936061, 0.03182118, 0.0339..."
2,How to make omette,with eggs and milk. Cook for 3 minutes,"[0.011496763, -0.008059607, 0.042288475, 0.049..."
3,how can I make a soup,Put tomato and peper and boil it,"[-0.018857786, -0.04761479, 0.036412578, 0.021..."
4,best way of making cake,heat the oven to 200 degrees. Use eggs oil and...,"[-0.002928722, -0.028932258, 0.060859833, 0.04..."
5,What is tips for making cakes?,You need to heat the oven before prepairign th...,"[0.015964048, -0.044111, 0.037022192, 0.035130..."
6,I want to make a cake with lemon,"Put sugar, lemon, egg and oil in a cup and coo...","[-0.02594058, -0.049454514, 0.0245295, 0.03507..."


In [23]:
query = "What is the right way of making cake?"
query_embeddings = embedding_model.encode(query)
print(query_embeddings.shape)

(768,)


In [24]:
# Cosine Similarity
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin


cos_sim_array = cosine_similarity([query_embeddings],
                                  list(df.embeddings.values))

In [25]:
cos_sim_array

array([[0.9181107 , 0.87832665, 0.8322697 , 0.7576039 , 0.93818796,
        0.9327589 , 0.8403721 ]], dtype=float32)

In [26]:
index_doc_cosine = np.argmax(cos_sim_array)

# Get the indices that would sort the array in ascending order
sorted_indices = np.argsort(cos_sim_array)
print(list(sorted_indices[0])[-3:])
similar_indexes = list(sorted_indices[0])[-3:]

[0, 5, 4]


In [27]:
similar_question = df.Question.loc[index_doc_cosine]
similar_answer = df.Answer.loc[index_doc_cosine]

## Most similar answer -> Single context

In [28]:
# Context
context = "Question: " + similar_question +\
"\n Answer: " + similar_answer

In [39]:
# Promopt

prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """
prompt

'Here is the context: Question: best way of making cake\n Answer: heat the oven to 200 degrees. Use eggs oil and milk\n             Using the relevant information from the context,\n             provide an answer to the query: What is the right way of making cake?."\n             If the context doesn\'t provide              any relevant information,              answer with              [I couldn\'t find a good match in the              document database for your query]\n             '

In [40]:
# Example of using T5 with Hugging Face Transformers library
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# For embeddings
promptinputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

# For text generation
outputs = model.generate(promptinputs.input_ids, do_sample=True, temperature = 0.9)
generated_text = tokenizer.decode(outputs[0])
generated_text

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad> A cake was made in a small oven</s>'

## Top 3 similar answer -> Multi context

In [41]:
# Multiple example
multiple_context = ""
for idx in similar_indexes:
  similar_question = df.Question.loc[idx]
  similar_answer = df.Answer.loc[idx]
  multiple_context += "Question: " + similar_question +\
"\n Answer: " + similar_answer +"\n"

multiple_context

'Question: How to make a cake\n Answer: Put Eggs and oil in a cup and cook them in oven for 30 minutes\nQuestion: What is tips for making cakes?\n Answer: You need to heat the oven before prepairign the ingredients.\nQuestion: best way of making cake\n Answer: heat the oven to 200 degrees. Use eggs oil and milk\n'

In [42]:
# Promopt

prompt_2 = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """
prompt_2

'Here is the context: Question: best way of making cake\n Answer: heat the oven to 200 degrees. Use eggs oil and milk\n             Using the relevant information from the context,\n             provide an answer to the query: What is the right way of making cake?."\n             If the context doesn\'t provide              any relevant information,              answer with              [I couldn\'t find a good match in the              document database for your query]\n             '

In [43]:
# Example of using T5 with Hugging Face Transformers library
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# For embeddings
promptinputs2 = tokenizer(prompt_2, return_tensors="pt", padding=True, truncation=True)

# For text generation
outputs = model.generate(promptinputs2.input_ids, do_sample=True, temperature = 0.9)
generated_text = tokenizer.decode(outputs[0])
generated_text

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad> Bake the cake at 200 degrees</s>'

## Top 3 similar answer -> Multi context v2

In [45]:
# Multiple example
multiple_context = []
for idx in similar_indexes:
  similar_answer = df.Answer.loc[idx]
  multiple_context.append(similar_answer)

multiple_context

['Put Eggs and oil in a cup and cook them in oven for 30 minutes',
 'You need to heat the oven before prepairign the ingredients.',
 'heat the oven to 200 degrees. Use eggs oil and milk']

In [46]:
# Promopt

prompt_3 = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """
prompt_3


# Example of using T5 with Hugging Face Transformers library
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# For embeddings
promptinputs3 = tokenizer(prompt_3, return_tensors="pt", padding=True, truncation=True)

# For text generation
outputs = model.generate(promptinputs2.input_ids, do_sample=True, temperature = 0.9)
generated_text = tokenizer.decode(outputs[0])
generated_text

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad> Answer: have a 200 degree oven</s>'

# Example for T5 search ask

In [16]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)

# Define title, context and question
title = "Gender Equality Efforts"
context = (
    "Gender equality is a vital societal goal that encompasses a wide array of studies and research efforts.",
    "These studies investigate various aspects of gender disparities, including the gender wage gap, representation in leadership roles, violence against women, healthcare access, and the impact of societal norms and media representations on gender roles.",
    "Researchers employ diverse methodologies, from quantitative data analysis to qualitative sociocultural examinations, to shed light on these complex issues.",
    "The findings from these studies serve as a crucial foundation for advocacy, policy changes, and social awareness campaigns, ultimately striving to create a more equitable and inclusive world where all individuals, regardless of their gender, can enjoy equal rights and opportunities."
)
question = "What are the studies for gender equality?"

# Format the prompt using the chosen format
prompt = f"Please answer a question about the following article about {title}:\n\n{context}\n\n{question}"

# Encode the prompt to tensor
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

# Generate a response using the specified parameters
output = model.generate(
    input_ids,
    do_sample=True,
    top_k=10,
    top_p=0.9,
    temperature=1,
    max_length=1000,
)

# Decode the generated text back to string
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

This article addresses the following questions: ('Gender equality is a vital societal goal that encompasses a wide array of studies and research efforts.') What are the studies for gender equality?
